In [ ]:
!pip install ktrain

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.3/25.3 MB 47.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 KB 54.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.4/265.4 KB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 69.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 KB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━

In [ ]:
import numpy as np
import ktrain
from ktrain import text
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv("/content/hi1.csv")
df.head()

FileNotFoundError: ignored

In [ ]:
df = df.drop(['Unnamed: 0','Unnamed: 0.1'],axis = 1)
df.head()

In [ ]:
df.replace("Negative",0,inplace=True)
df.replace("Positive",1,inplace=True)
df.replace("Mixed feelings",2,inplace=True)
df.replace("unknown state",3,inplace=True)
df.head()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df['Translated_text'], df['Sentiment'], shuffle=True, test_size = 0.2, random_state=45, stratify=df['Sentiment'])

In [ ]:
x_train = x_train.to_numpy()
y_train = y_train.to_numpy()
#x_test = x_test.to_numpy()
#y_test = y_test.to_numpy()

# BERT

In [ ]:
(x_train_bert,y_train_bert) , (x_val_bert,y_val_bert),preproc = text.texts_from_array(x_train = x_train,y_train = y_train,class_names = ["0","1","2","3"],preprocess_mode='bert',lang = 'kn',maxlen = 65,max_features=35000)

In [ ]:
model = text.text_classifier('bert',train_data = (x_train_bert,y_train_bert),preproc = preproc)
learner = ktrain.get_learner(model,train_data=(x_train_bert,y_train_bert),val_data=(x_val_bert,y_val_bert),batch_size = 16)

In [ ]:
learner.lr_find()

In [ ]:
learner.lr_plot()

In [ ]:
learner.autofit(1e-3)

## Prediction

In [ ]:
predictor = ktrain.get_predictor(learner.model,preproc)

In [ ]:
test_df = pd.concat([x_test,y_test],axis=1)
test_df.head()

In [ ]:
test_df['Predicted_Sentiment'] = predictor.predict(test_df['Translated_text'].to_list())
test_df.head()

In [ ]:
test_df["Sentiment"] = pd.to_numeric(test_df["Sentiment"])
test_df["Predicted_Sentiment"] = pd.to_numeric(test_df["Predicted_Sentiment"])

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score,confusion_matrix
cnf_matrix = confusion_matrix(test_df['Sentiment'],test_df['Predicted_Sentiment'])
cnf_matrix

In [ ]:
acc = accuracy_score(test_df['Sentiment'],test_df['Predicted_Sentiment'])
macro_prec = precision_score(test_df['Sentiment'],test_df['Predicted_Sentiment'],average = 'macro')
macro_rec = recall_score(test_df['Sentiment'],test_df['Predicted_Sentiment'],average='macro')
macro_f1 = f1_score(test_df['Sentiment'],test_df['Predicted_Sentiment'],average='macro')
print('accuracy:%0.4f'%acc,'\tprecision:%0.4f'%macro_prec,'\trecall:%0.4f'%macro_rec,'\tF1-score:%0.4f'%macro_f1)

In [ ]:
acc = accuracy_score(test_df['Sentiment'],test_df['Predicted_Sentiment'])
wt_prec = precision_score(test_df['Sentiment'],test_df['Predicted_Sentiment'],average = 'weighted')
wt_rec = recall_score(test_df['Sentiment'],test_df['Predicted_Sentiment'],average='weighted')
wt_f1 = f1_score(test_df['Sentiment'],test_df['Predicted_Sentiment'],average='weighted')
print('accuracy:%0.4f'%acc,'\tprecision:%0.4f'%wt_prec,'\trecall:%0.4f'%wt_rec,'\tF1-score:%0.4f'%wt_f1)

In [ ]:
test_df.head()

In [ ]:
test_df.to_csv('hi.csv')